# Drzewo decyzyjne

In [173]:
from sklearn.model_selection import cross_val_score, cross_validate, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import pandas as pd
from IPython.display import display

In [80]:
import csv
import numpy as np

class IrisData:
    def __init__(self, data, target, labels):
        self.data = data
        self.target = target
        self.labels = labels

def load_iris_data():
    data = []
    labels = []
    target = []

    with open('./iris/iris.data', newline='') as csvfile:
        data_reader = csv.reader(csvfile, delimiter=',')
        for row in data_reader:
            if len(row) != 0:
                if row[-1] not in labels:
                    labels.append(row[-1])
                
                target.append(labels.index(row[-1]))
                data.append([float(x) for x in row[:-1]])

    data = np.array(data)
    target = np.array(target)
    labels = np.array(labels)
    
    return IrisData(data, target, labels)

In [81]:
iris_data = load_iris_data()
print(iris_data.labels)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [82]:
clf = DecisionTreeClassifier(random_state=0)
print((iris_data.data[10]))
cross_val_score(clf, iris_data.data, iris_data.target, cv=10)

[5.4 3.7 1.5 0.2]


array([1.        , 0.93333333, 1.        , 0.93333333, 0.93333333,
       0.86666667, 0.93333333, 1.        , 1.        , 1.        ])

In [174]:
def conduct_tests_tree(folds, scores_dict, random_states, param_name = None, param_value = None):
    res = {score_name: [] for score_name in scores_dict.keys()}

    for random_state in random_states:
        if param_name is None:
            clf = DecisionTreeClassifier(random_state=random_state)
        else:
            clf = DecisionTreeClassifier(random_state=random_state, **{param_name: param_value})
        kf = KFold(n_splits=folds, shuffle=True, random_state=random_state)
        results = cross_validate(clf, iris_data.data, iris_data.target, scoring=scores_dict,
                                cv=kf)
        for score_name, score_values in results.items():
            if 'test_' in score_name:
                res[score_name[5:]].append(score_values)

    avg_scores = {}
    std_dev_scores = {}

    for score_name, score_values in res.items():
        scores_array = np.concatenate(score_values)
        avg_scores[score_name] = np.mean(scores_array)
        std_dev_scores[score_name] = np.std(scores_array)

    return avg_scores, std_dev_scores

In [84]:
RANDOM_STATES = [6, 5, 2024]
FOLDS_N = 5
SCORES_DICT = {'accuracy': 'accuracy',
                'precision': 'precision_macro', 
                'recall': 'recall_macro',
                'f1': 'f1_macro'}

Testing for default

In [175]:
test_res = conduct_tests_tree(FOLDS_N, SCORES_DICT, RANDOM_STATES)

for score_name in SCORES_DICT.keys():
    print(f"Average {score_name.capitalize()}: {test_res[0][score_name]}")
    print(f"Standard Deviation of {score_name.capitalize()}: {test_res[1][score_name]}")

Average Accuracy: 0.9422222222222223
Standard Deviation of Accuracy: 0.044666113871648386
Average Precision: 0.9432556332556333
Standard Deviation of Precision: 0.0443131404597508
Average Recall: 0.9436294569627902
Standard Deviation of Recall: 0.044999873947333
Average F1: 0.9403874222658266
Standard Deviation of F1: 0.04647450346863861


In [176]:
PARAMETERS = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [1, 2, 3, 4, 5, None]
}

results_by_param = {}

for param_name, param_values in PARAMETERS.items():
    results_for_param = []
    for param_value in param_values:
        avg_scores, std_dev_scores = conduct_tests_tree(FOLDS_N, SCORES_DICT, RANDOM_STATES, param_name, param_value)
        for metric in avg_scores.keys():
            results_for_param.append({
                #'Param Name': param_name,
                'Wartość parametru': param_value,
                'Miary jakości': metric,
                'Średnia': avg_scores[metric],
                'Odch. std.': std_dev_scores[metric]
            })

    results_df = pd.DataFrame(results_for_param)


    results_df = results_df.pivot_table(index=['Wartość parametru'], columns='Miary jakości')
    
    results_df.columns = results_df.columns.swaplevel(0, 1)
    results_df.sort_index(axis=1, level=0, inplace=True)

    results_by_param[param_name] = results_df

    results_df = results_df.map(lambda x: '{:.3f}'.format(x).replace('.', ','))

    display(results_df)

    latex_code = results_df.to_latex(multicolumn_format='c')

    print(latex_code)

        

Miary jakości       accuracy                 f1          precision          \
                  Odch. std. Średnia Odch. std. Średnia Odch. std. Średnia   
Wartość parametru                                                            
entropy                0,047   0,933      0,053   0,931      0,051   0,933   
gini                   0,045   0,942      0,046   0,940      0,044   0,943   

Miary jakości         recall          
                  Odch. std. Średnia  
Wartość parametru                     
entropy                0,052   0,934  
gini                   0,045   0,944

\begin{tabular}{lllllllll}
\toprule
Miary jakości & \multicolumn{2}{c}{accuracy} & \multicolumn{2}{c}{f1} & \multicolumn{2}{c}{precision} & \multicolumn{2}{c}{recall} \\
 & Odch. std. & Średnia & Odch. std. & Średnia & Odch. std. & Średnia & Odch. std. & Średnia \\
Wartość parametru &  &  &  &  &  &  &  &  \\
\midrule
entropy & 0,047 & 0,933 & 0,053 & 0,931 & 0,051 & 0,933 & 0,052 & 0,934 \\
gini & 0,045 & 0,942 & 0,046 & 0,940 & 0,044 & 0,943 & 0,045 & 0,944 \\
\bottomrule
\end{tabular}



Miary jakości       accuracy                 f1          precision          \
                  Odch. std. Średnia Odch. std. Średnia Odch. std. Średnia   
Wartość parametru                                                            
best                   0,045   0,942      0,046   0,940      0,044   0,943   
random                 0,031   0,956      0,035   0,955      0,035   0,955   

Miary jakości         recall          
                  Odch. std. Średnia  
Wartość parametru                     
best                   0,045   0,944  
random                 0,033   0,959

\begin{tabular}{lllllllll}
\toprule
Miary jakości & \multicolumn{2}{c}{accuracy} & \multicolumn{2}{c}{f1} & \multicolumn{2}{c}{precision} & \multicolumn{2}{c}{recall} \\
 & Odch. std. & Średnia & Odch. std. & Średnia & Odch. std. & Średnia & Odch. std. & Średnia \\
Wartość parametru &  &  &  &  &  &  &  &  \\
\midrule
best & 0,045 & 0,942 & 0,046 & 0,940 & 0,044 & 0,943 & 0,045 & 0,944 \\
random & 0,031 & 0,956 & 0,035 & 0,955 & 0,035 & 0,955 & 0,033 & 0,959 \\
\bottomrule
\end{tabular}



c:\Users\Kuba\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kuba\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kuba\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

Miary jakości       accuracy                 f1          precision          \
                  Odch. std. Średnia Odch. std. Średnia Odch. std. Średnia   
Wartość parametru                                                            
1.0                    0,034   0,633      0,016   0,539      0,016   0,482   
2.0                    0,042   0,938      0,045   0,935      0,043   0,939   
3.0                    0,040   0,938      0,044   0,935      0,041   0,938   
4.0                    0,041   0,936      0,042   0,934      0,041   0,937   
5.0                    0,041   0,936      0,042   0,934      0,041   0,937   

Miary jakości         recall          
                  Odch. std. Średnia  
Wartość parametru                     
1.0                    0,000   0,667  
2.0                    0,045   0,938  
3.0                    0,044   0,937  
4.0                    0,041   0,937  
5.0                    0,041   0,937

\begin{tabular}{lllllllll}
\toprule
Miary jakości & \multicolumn{2}{c}{accuracy} & \multicolumn{2}{c}{f1} & \multicolumn{2}{c}{precision} & \multicolumn{2}{c}{recall} \\
 & Odch. std. & Średnia & Odch. std. & Średnia & Odch. std. & Średnia & Odch. std. & Średnia \\
Wartość parametru &  &  &  &  &  &  &  &  \\
\midrule
1.000000 & 0,034 & 0,633 & 0,016 & 0,539 & 0,016 & 0,482 & 0,000 & 0,667 \\
2.000000 & 0,042 & 0,938 & 0,045 & 0,935 & 0,043 & 0,939 & 0,045 & 0,938 \\
3.000000 & 0,040 & 0,938 & 0,044 & 0,935 & 0,041 & 0,938 & 0,044 & 0,937 \\
4.000000 & 0,041 & 0,936 & 0,042 & 0,934 & 0,041 & 0,937 & 0,041 & 0,937 \\
5.000000 & 0,041 & 0,936 & 0,042 & 0,934 & 0,041 & 0,937 & 0,041 & 0,937 \\
\bottomrule
\end{tabular}



In [177]:
def conduct_tests_svm(folds, scores_dict, random_states, param_name = None, param_value = None):
    res = {score_name: [] for score_name in scores_dict.keys()}

    for random_state in random_states:
        if param_name is None:
            clf = SVC(random_state=random_state)
        else:
            clf = SVC(random_state=random_state, **{param_name: param_value}, tol=10e-6)
        kf = KFold(n_splits=folds, shuffle=True, random_state=random_state)
        results = cross_validate(clf, iris_data.data, iris_data.target, scoring=scores_dict,
                                cv=kf)
        for score_name, score_values in results.items():
            if 'test_' in score_name:
                res[score_name[5:]].append(score_values)

    avg_scores = {}
    std_dev_scores = {}

    for score_name, score_values in res.items():
        scores_array = np.concatenate(score_values)
        avg_scores[score_name] = np.mean(scores_array)
        std_dev_scores[score_name] = np.std(scores_array)

    return avg_scores, std_dev_scores

In [178]:
test_res = conduct_tests_svm(FOLDS_N, SCORES_DICT, RANDOM_STATES)

for score_name in SCORES_DICT.keys():
    print(f"Average {score_name.capitalize()}: {test_res[0][score_name]}")
    print(f"Standard Deviation of {score_name.capitalize()}: {test_res[1][score_name]}")

Average Accuracy: 0.9622222222222222
Standard Deviation of Accuracy: 0.0341384255460827
Average Precision: 0.9613583330249997
Standard Deviation of Precision: 0.03543533171641029
Average Recall: 0.9623702840369507
Standard Deviation of Recall: 0.03536243510349699
Average F1: 0.9601063605816151
Standard Deviation of F1: 0.0364433482627812


In [181]:
PARAMETERS = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'max_iter': np.arange(1, 16, 1)
}

results_by_param = {}

for param_name, param_values in PARAMETERS.items():
    results_for_param = []
    for param_value in param_values:
        avg_scores, std_dev_scores = conduct_tests_svm(FOLDS_N, SCORES_DICT, RANDOM_STATES, param_name, param_value)
        for metric in avg_scores.keys():
            results_for_param.append({
                #'Param Name': param_name,
                'Wartość parametru': param_value,
                'Miary jakości': metric,
                'Średnia': avg_scores[metric],
                'Odch. std.': std_dev_scores[metric]
            })

    results_df = pd.DataFrame(results_for_param)


    results_df = results_df.pivot_table(index=['Wartość parametru'], columns='Miary jakości')
    
    results_df.columns = results_df.columns.swaplevel(0, 1)
    results_df.sort_index(axis=1, level=0, inplace=True)

    results_by_param[param_name] = results_df

    results_df = results_df.map(lambda x: '{:.3f}'.format(x).replace('.', ','))

    display(results_df)

    latex_code = results_df.to_latex(multicolumn_format='c')

    print(latex_code)

Miary jakości       accuracy                 f1          precision          \
                  Odch. std. Średnia Odch. std. Średnia Odch. std. Średnia   
Wartość parametru                                                            
0.1                    0,042   0,913      0,047   0,909      0,037   0,921   
1.0                    0,034   0,962      0,036   0,960      0,035   0,961   
10.0                   0,025   0,973      0,028   0,971      0,028   0,972   

Miary jakości         recall          
                  Odch. std. Średnia  
Wartość parametru                     
0.1                    0,040   0,918  
1.0                    0,035   0,962  
10.0                   0,027   0,974

\begin{tabular}{lllllllll}
\toprule
Miary jakości & \multicolumn{2}{c}{accuracy} & \multicolumn{2}{c}{f1} & \multicolumn{2}{c}{precision} & \multicolumn{2}{c}{recall} \\
 & Odch. std. & Średnia & Odch. std. & Średnia & Odch. std. & Średnia & Odch. std. & Średnia \\
Wartość parametru &  &  &  &  &  &  &  &  \\
\midrule
0.100000 & 0,042 & 0,913 & 0,047 & 0,909 & 0,037 & 0,921 & 0,040 & 0,918 \\
1.000000 & 0,034 & 0,962 & 0,036 & 0,960 & 0,035 & 0,961 & 0,035 & 0,962 \\
10.000000 & 0,025 & 0,973 & 0,028 & 0,971 & 0,028 & 0,972 & 0,027 & 0,974 \\
\bottomrule
\end{tabular}



c:\Users\Kuba\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kuba\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kuba\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

Miary jakości       accuracy                 f1          precision          \
                  Odch. std. Średnia Odch. std. Średnia Odch. std. Średnia   
Wartość parametru                                                            
linear                 0,024   0,980      0,025   0,979      0,024   0,979   
poly                   0,024   0,967      0,027   0,965      0,027   0,966   
rbf                    0,034   0,962      0,036   0,960      0,035   0,961   
sigmoid                0,066   0,242      0,032   0,128      0,021   0,081   

Miary jakości         recall          
                  Odch. std. Średnia  
Wartość parametru                     
linear                 0,024   0,981  
poly                   0,025   0,968  
rbf                    0,035   0,962  
sigmoid                0,074   0,314

c:\Users\Kuba\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\Kuba\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\Kuba\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\Kuba\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\Kuba\AppDat

\begin{tabular}{lllllllll}
\toprule
Miary jakości & \multicolumn{2}{c}{accuracy} & \multicolumn{2}{c}{f1} & \multicolumn{2}{c}{precision} & \multicolumn{2}{c}{recall} \\
 & Odch. std. & Średnia & Odch. std. & Średnia & Odch. std. & Średnia & Odch. std. & Średnia \\
Wartość parametru &  &  &  &  &  &  &  &  \\
\midrule
linear & 0,024 & 0,980 & 0,025 & 0,979 & 0,024 & 0,979 & 0,024 & 0,981 \\
poly & 0,024 & 0,967 & 0,027 & 0,965 & 0,027 & 0,966 & 0,025 & 0,968 \\
rbf & 0,034 & 0,962 & 0,036 & 0,960 & 0,035 & 0,961 & 0,035 & 0,962 \\
sigmoid & 0,066 & 0,242 & 0,032 & 0,128 & 0,021 & 0,081 & 0,074 & 0,314 \\
\bottomrule
\end{tabular}



c:\Users\Kuba\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\Kuba\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\Kuba\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\Kuba\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=2).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\Kuba\AppDat

Miary jakości       accuracy                 f1          precision          \
                  Odch. std. Średnia Odch. std. Średnia Odch. std. Średnia   
Wartość parametru                                                            
1                      0,047   0,911      0,053   0,907      0,043   0,922   
2                      0,037   0,942      0,040   0,940      0,033   0,946   
3                      0,055   0,927      0,067   0,922      0,045   0,936   
4                      0,043   0,944      0,049   0,941      0,039   0,949   
5                      0,031   0,956      0,035   0,953      0,030   0,957   
6                      0,036   0,947      0,041   0,943      0,034   0,950   
7                      0,024   0,971      0,026   0,970      0,026   0,970   
8                      0,034   0,967      0,040   0,964      0,035   0,967   
9                      0,024   0,980      0,026   0,979      0,025   0,980   
10                     0,030   0,973      0,033   0,972      0,033   0,972   
11                     0,025   0,973      0,027   0,972      0,027   0,973   
12                     0,025   0,973      0,027   0,972      0,027   0,973   
13                     0,023   0,978      0,024   0,977      0,024   0,977   
14                     0,033   0,969      0,034   0,968      0,032   0,969   
15                     0,034   0,971      0,037   0,970      0,036   0,970   

Miary jakości         recall          
                  Odch. std. Średnia  
Wartość parametru                     
1                      0,044   0,915  
2                      0,033   0,945  
3                      0,052   0,930  
4                      0,039   0,947  
5                      0,031   0,956  
6                      0,035   0,948  
7                      0,024   0,973  
8                      0,032   0,969  
9                      0,023   0,980  
10                     0,031   0,974  
11                     0,025   0,974  
12                     0,025   0,974  
13                     0,023   0,979  
14                     0,033   0,971  
15                     0,036   0,972

\begin{tabular}{lllllllll}
\toprule
Miary jakości & \multicolumn{2}{c}{accuracy} & \multicolumn{2}{c}{f1} & \multicolumn{2}{c}{precision} & \multicolumn{2}{c}{recall} \\
 & Odch. std. & Średnia & Odch. std. & Średnia & Odch. std. & Średnia & Odch. std. & Średnia \\
Wartość parametru &  &  &  &  &  &  &  &  \\
\midrule
1 & 0,047 & 0,911 & 0,053 & 0,907 & 0,043 & 0,922 & 0,044 & 0,915 \\
2 & 0,037 & 0,942 & 0,040 & 0,940 & 0,033 & 0,946 & 0,033 & 0,945 \\
3 & 0,055 & 0,927 & 0,067 & 0,922 & 0,045 & 0,936 & 0,052 & 0,930 \\
4 & 0,043 & 0,944 & 0,049 & 0,941 & 0,039 & 0,949 & 0,039 & 0,947 \\
5 & 0,031 & 0,956 & 0,035 & 0,953 & 0,030 & 0,957 & 0,031 & 0,956 \\
6 & 0,036 & 0,947 & 0,041 & 0,943 & 0,034 & 0,950 & 0,035 & 0,948 \\
7 & 0,024 & 0,971 & 0,026 & 0,970 & 0,026 & 0,970 & 0,024 & 0,973 \\
8 & 0,034 & 0,967 & 0,040 & 0,964 & 0,035 & 0,967 & 0,032 & 0,969 \\
9 & 0,024 & 0,980 & 0,026 & 0,979 & 0,025 & 0,980 & 0,023 & 0,980 \\
10 & 0,030 & 0,973 & 0,033 & 0,972 & 0,033 & 0,972 & 0,031 